**Описание работы:** были обучены уни-, би- и три-граммные модели на корпусе 4-х томов романа "Война и мир" с предварительным подбором токенизатора, а также 4 способа генерации текстов. Было получено: лучшая модель – биграммная, лучший способ генерации – рандомный выбор следующего токена с и без beam search.

# Creation of corpora

Тестовые предложения использовались, чтобы отдебажить работу методов, тестовый подкорпус (~1-я часть 1-го тома "Войны и мира") использовался для подбора гиперпараметров, а весь корпус (все 4 тома ВиМ) использовался для финальной оценки модели и генерации.

### Test sentences:

In [9]:
from src.corpus import Coprus
t_wap_corpus = Coprus('test_sentences.txt')
t_train, t_test = t_wap_corpus.split_corpus(0.7)

In [10]:
t_train

'Анна Павловна Шерер, напротив, несмотря на свои сорок лет, была преисполнена оживления и порывов.\nБыть энтузиасткой сделалось ее общественным положением, и иногда, когда ей даже того не хотелось, она, чтобы не обмануть ожиданий людей, знавших ее, делалась энтузиасткой. Сдержанная улыбка, игравшая постоянно на лице Анны Павловны, хотя и не шла к ее отжившим чертам, выражала, как у избалованных детей, постоянное с'

### Test corpus:

In [16]:
from src.corpus import Coprus
wap_testing_corpus = Coprus('test.txt')
testing_train, testing_test = wap_testing_corpus.split_corpus(0.7)

### Corpus:

In [3]:
from src.corpus import Coprus
wap_corpus = Coprus('voina_i_mir.txt')
train, test = wap_corpus.split_corpus(0.7)

# Testing code

In [11]:
from src.model import Tokenizer, NgramModel

In [12]:
t_tokenizer = Tokenizer(tokenization_mode='with_tag', register='lower')
t_train_tokens = t_tokenizer.tokenize(t_train)
t_test_tokens = t_tokenizer.tokenize(t_test)

In [14]:
test_sent_model = NgramModel(n=2, smoothing_algorithm='laplace')
test_sent_model.train(t_train_tokens)
test_sent_PP = test_sent_model.evaluate(t_test_tokens)

100%|██████████| 53/53 [00:00<?, ?it/s]


In [15]:
test_sent_PP

21.619620022596028

# Hyperparameters tuning on test corpus

In [4]:
from src.model import Tokenizer, NgramModel

Сначала протестируем кросс-валидацией все типы токенизаторов и регистров на тестовом подкорпусе, обучая биграммную модель с сглаживанием Лапласа.

In [17]:
tokenizers = ['split', 'pymorphy', 'with_punctuation', 'with_selected_punctuation', 'with_tag']
registers = ['both', 'lower']

In [18]:
num_iter = 1
for tokenizer in tokenizers:
    for register in registers:
        testing_tokenizer = Tokenizer(tokenization_mode=tokenizer, register=register)
        testing_train_tokens = testing_tokenizer.tokenize(testing_train)
        testing_test_tokens = testing_tokenizer.tokenize(testing_test)

        testing_model = NgramModel(n=2, smoothing_algorithm='laplace')
        testing_model.train(testing_train_tokens)
        testing_PP = testing_model.evaluate(testing_test_tokens)
        
        print(f'Model No {num_iter} -- tokenizer: {tokenizer} -- register: {register}')
        print(f'Model perplexity: {testing_PP}')
        num_iter += 1

100%|██████████| 17247/17247 [00:00<00:00, 595232.17it/s]


Model No 1 -- tokenizer: split -- register: both
Model perplexity: 2003.4209058468625


100%|██████████| 16891/16891 [00:00<00:00, 857263.73it/s]


Model No 2 -- tokenizer: split -- register: lower
Model perplexity: 1754.9741851201345


100%|██████████| 16560/16560 [00:00<00:00, 535645.40it/s]


...

Model No 8 -- tokenizer: with_selected_punctuation -- register: lower
Model perplexity: 383.0875250418025


100%|██████████| 16635/16635 [00:00<00:00, 758130.29it/s]


Model No 9 -- tokenizer: with_tag -- register: both
Model perplexity: 431.6042854933508


100%|██████████| 16196/16196 [00:00<00:00, 648134.22it/s]

Model No 10 -- tokenizer: with_tag -- register: lower
Model perplexity: 372.01582691543894


Топ-3 моделей:

1. Model No 4 -- tokenizer: pymorphy -- register: lower.   
       Model perplexity: 361.8203557090759
2. Model No 10 -- tokenizer: with_tag -- register: lower.   
       Model perplexity: 372.01582691543894
3. Model No 8 -- tokenizer: with_selected_punctuation -- register: lower.     
    Model perplexity: 383.0875250418025

В дальнейшем будем использовать токенизатор лучшей модели – pymorphy. Однако стоит отметить, что на всём нашем корпусе могли быть даны другие результаты, отличные от того, что получено на подкорпусе. Но ради экономии времени (так как корпус очень большой) пренебрежём этим.

# Training and evaluation on test corpus

In [2]:
from src.model import Tokenizer, NgramModel

Подбирать оптимальное n я решил на всём корпусе (у меня есть подозрение, что обученная на подкорпусе модель не будет качественно репрезентировать отношения между словами во всём корпусе). 

In [20]:
n = 1

tokenizer = Tokenizer(tokenization_mode='pymorphy', register='lower')
train_tokens = tokenizer.tokenize(train)
test_tokens = tokenizer.tokenize(test)

n1model = NgramModel(n=n, smoothing_algorithm='laplace')
n1model.train(train_tokens)
n1_PP = n1model.evaluate(test_tokens)

print(f'Model No 1 -- n: {n}')
print(f'Model perplexity: {n1_PP}')

100%|██████████| 25164/25164 [00:00<00:00, 813439.99it/s]

Model No 1 -- n: 1
Model perplexity: 5.448979736066591


In [5]:
n = 2

tokenizer = Tokenizer(tokenization_mode='pymorphy', register='lower')
train_tokens = tokenizer.tokenize(train)
test_tokens = tokenizer.tokenize(test)

n2model = NgramModel(n=n, smoothing_algorithm='laplace')
n2model.train(train_tokens)
n2_PP = n2model.evaluate(test_tokens)

print(f'Model No 2 -- n: {n}')
print(f'Model perplexity: {n2_PP}')

100%|██████████| 121224/121224 [00:00<00:00, 586499.29it/s]

Model No 2 -- n: 2
Model perplexity: 231.43349155465745


In [9]:
n = 3

tokenizer = Tokenizer(tokenization_mode='pymorphy', register='lower')
train_tokens = tokenizer.tokenize(train)
test_tokens = tokenizer.tokenize(test)

n3model = NgramModel(n=n, smoothing_algorithm='laplace')
n3model.train(train_tokens)
n3_PP = n3model.evaluate(test_tokens)

print(f'Model No 3 -- n: {n}')
print(f'Model perplexity: {n3_PP}')

100%|██████████| 269078/269078 [00:00<00:00, 727162.16it/s]


Model No 1 -- n: 3
Model perplexity: 5170.761327431317


Если не брать во внимание, что перплексия, видимо, сильно зависит от объёма корпуса и количества н-грамм, а также в тексте ВиМ, как я подозреваю, очень много уникальных нграмм, то мы видим, что наименьшее значение этой метрики у первой модели. Однако столь низкое значение, как мне кажется, скорее сигнализирует о том, что модель плохо "выучила" зависимости между словами в корпусе и поэтому способна "выбирать" только из малого количества токенов. Поэтому я считаю оптимальной биграммную модель – причём как по качеству, так и по времени обучения. Дальше предсказания будем строить именно на её основе.

Сохраним словарь вероятностей лучшей модели.

In [6]:
import pickle

with open('ngrams_probabilities.pickle', 'wb') as file:
    pickle.dump(n2model.ngrams_probabilities, file, protocol=pickle.HIGHEST_PROTOCOL)

# Generation of texts

In [2]:
import pickle
with open('ngrams_probabilities.pickle', 'rb') as file:
    probs_dict = pickle.load(file)

In [2]:
probs_dict

{('часть',): 4.345813786225074e-06,
 ('часть', 'первая'): 4.6724605177086255e-05,
 ('первая',): 4.345813786225074e-06,
 ('первая', '.'): 4.6724605177086255e-05,
 ('.',): 4.345813786225074e-06,
 ('.', 'i'): 4.6724605177086255e-05,
 ('i',): 4.345813786225074e-06,
 ('i', '.'): 4.6724605177086255e-05,
 ('.', '—'): 4.6724605177086255e-05,
 ('—',): 4.345813786225074e-06,
 ('—', 'eh'): 4.6724605177086255e-05,
 ('eh',): 4.345813786225074e-06,
 ('eh', 'bien'): 4.6724605177086255e-05,
 ('bien',): 4.345813786225074e-06,
 ('bien', ','): 4.6724605177086255e-05,
 (',',): 4.345813786225074e-06,
 (',', 'mon'): 4.6724605177086255e-05,
 ('mon',): 4.345813786225074e-06,
 ('mon', 'prince'): 4.6724605177086255e-05,
 ('prince',): 4.345813786225074e-06,
 ('prince', '.'): 4.6724605177086255e-05,
 ('.', 'gênes'): 4.6724605177086255e-05,
 ('gênes',): 4.345813786225074e-06,
 ('gênes', 'et'): 4.6724605177086255e-05,
 ('et',): 4.345813786225074e-06,
 ('et', 'lucques'): 4.6724605177086255e-05,
 ('lucques',): 4.3458

In [3]:
from src.model import NgramModel
n2model = NgramModel(n=2, smoothing_algorithm='laplace')
n2model.ngrams_probabilities = probs_dict

### 1. Standart generation

Стандартный способ генерации текстов: последовательно подбираются наиболее вероятные нграммы, начинающиеся с последнего слова предыдущей нграммы

In [3]:
n2model.generate_text(first_word='мне', len_text=20, generation_mode='most_probable')

'мне не скажете , mon prince . i . i . i . i . i . i . i .'

In [4]:
n2model.generate_text(first_word='вы', len_text=10, generation_mode='most_probable')

'вы мне не скажете , mon prince . i . i'

In [7]:
n2model.generate_text(first_word='наполеон', len_text=10, generation_mode='most_probable')

'наполеон случайно упал в июле 1805 года известная анна павловна шерер'

In [14]:
n2model.generate_text(first_word='андрей', len_text=10, generation_mode='most_probable')

'андрей болконский , mon prince . i . i . i'

In [18]:
# выберем рандомное слово из текста и построим предсказание для него
from random import randint
word = list(probs_dict.keys())[randint(0, len(probs_dict.keys()))][0]
word

'же'

In [19]:
n2model.generate_text(first_word=word, len_text=12, generation_mode='most_probable')

'же решили по привычке , mon prince . i . i . i'

In [9]:
# для несуществующего в трейне слова модель не выдаёт ошибку, а просто не делает предсказание
n2model.generate_text(first_word='ыарвпао', len_text=10, generation_mode='most_probable')

'ыарвпао'

Модель похожа на старенький т9 (ну, я полагаю, там нграммные модели и были): она выдаёт несколько подходящих предыдущему слову, но не контексту, токенов, а потом начинает поочерёдно выдавать два каких-то токена. Поэтому есть смысл опробовать другие алгоритмы генерации

### 2. Generation with random choice 

Способ генерации, похожий на предыдущий, однако вместо самого частотного выбирается случайный из топ k самых вероятных.

In [3]:
n2model.generate_text(first_word='мне', len_text=20, generation_mode='random_next_word', num_ngrams=4)

'мне не друг мой крест действительно вынимая руку анны семеновны , des imbéciles . — отвечал князь василий желал , mon'

In [5]:
n2model.generate_text(first_word='вы', len_text=10, generation_mode='random_next_word', num_ngrams=4)

'вы весь вечер ни в чулках , que nous allons doucement'

In [6]:
n2model.generate_text(first_word='вы', len_text=15, generation_mode='random_next_word', num_ngrams=10)

'вы дадите мне пишет . non avenu , que buonaparte a chi la manie des visites'

In [7]:
# выберем рандомное слово из текста и построим предсказание для него
from random import randint
word = list(probs_dict.keys())[randint(0, len(probs_dict.keys()))][0]
word

'отрывисто'

In [8]:
n2model.generate_text(first_word=word, len_text=30, generation_mode='random_next_word', num_ngrams=5)

'отрывисто смеяться самому понравилось офицеру во все , que toutes les délices de mieux dans le jour qui se met lui ferez déclarer , des gens est un pareil auditoire ,'

Генерация получается уже лучше, так как способны генерироваться текстовые отрывки большей длины без повторений.

### 3. Beam search generation

Этот способ подразумевает генерацию нескольких предложений небольшой длины, вероятности которых сравниваются, и на основе этого сравнения модель выдаёт наилучший результат.

In [4]:
n2model.generate_text(first_word='мне', len_text=5, generation_mode='beam_search', num_ngrams=6)

'мне не скажете , mon prince .'

In [5]:
n2model.generate_text(first_word='вы', len_text=5, generation_mode='beam_search', num_ngrams=6)

'вы мне не скажете , mon prince'

In [6]:
n2model.generate_text(first_word='красота', len_text=5, generation_mode='beam_search', num_ngrams=6)

'красота его , mon prince . i'

In [9]:
n2model.generate_text(first_word='наполеон', len_text=10, generation_mode='beam_search', num_ngrams=6)

'наполеон случайно упал в июле 1805 года известная анна павловна шерер ,'

In [10]:
# выберем рандомное слово из текста и построим предсказание для него
from random import randint
word = list(probs_dict.keys())[randint(0, len(probs_dict.keys()))][0]
word

'долгое'

In [11]:
n2model.generate_text(first_word=word, len_text=10, generation_mode='beam_search', num_ngrams=6)

'долгое сосредоточение боя отдал бы знали , mon prince . i .'

Как ни странно, но beam search показал результат, близкий к генерации стандартным способом. Возможно, это связано с тем, что предложениями с наибольшей вероятностью оказывались те, в которых встречались наиболее вероятные для введённого слова биграммы.

### 4. Beam search generation with random choice

После анализа результатов генерации появилась идея, что наиболее эффективной будет генерация, совмещающая beam search со случайным выбором одного из наиболее вероятных токенов.

In [3]:
n2model.generate_text(first_word='вы', len_text=5, generation_mode='beam_search_with_random', num_ngrams=6)

{('мне', 'нужно', '.', 'i', 'k', 'l'): 2.2270409430120467e-22}


'вы мне нужно . i k l'

In [4]:
n2model.generate_text(first_word='вы', len_text=5, generation_mode='beam_search_with_random', num_ngrams=10)

'вы мне сто к хозяйке . 1'

In [5]:
n2model.generate_text(first_word='вы', len_text=15, generation_mode='beam_search_with_random', num_ngrams=6)

'вы мне не друг на лице фрейлины , des nations » не только что бог мой крест'

In [7]:
n2model.generate_text(first_word='мне', len_text=15, generation_mode='beam_search_with_random', num_ngrams=6)

'мне не скажете свои желания лизы опустилась . 1 ну что у нее . i . non'

In [8]:
n2model.generate_text(first_word='дуб', len_text=15, generation_mode='beam_search_with_random', num_ngrams=6)

'дуб и приближенная императрицы приглашал его отличали , je crois que buonaparte . — сказал бы вас'

In [9]:
n2model.generate_text(first_word='ростова', len_text=15, generation_mode='beam_search_with_random', num_ngrams=6)

'ростова , de mieux dans toute cette douce marie . je crois que toutes ces peines si'

In [10]:
# выберем рандомное слово из текста и построим предсказание для него
from random import randint
word = list(probs_dict.keys())[randint(0, len(probs_dict.keys()))][0]
word

'проходили'

In [11]:
n2model.generate_text(first_word=word, len_text=15, generation_mode='beam_search_with_random', num_ngrams=6)

'проходили с светлым , mon apprentissage de cet homme ? разве можно выразить . — eh bien'

В целом заметного улучшения качества по сравнению с обычным рандомным выбором не видно, но в целом этот способ действительно неплохой.

# Conclusion

В данной работе была написана нграммная модель с оценкой по перплексии. Сначала на тестовом подкорпусе были подобраны гиперпараметры – токенизатор и вид регистра – и получено, что лучший результат по перплексии даёт модель с токенизатором из библиотеки pymorphy и понижением регистра. Затем на основном обучающем корпусе были обучены уни-, би- и триграммы и после подсчёта перплексии на тестовом корпусе была выбрана биграммная модель. На её материале были протестированы различные способы генерации текстов: путём выбора самого вероятностного, выбора случайного слова из наиболее вероятных, beam search и его вариация с выбором случайных вероятностных слов. Мы можем наблюдать, что модель генерирует наиболее адекватные тексты с помощью методов, использующих случайный подбор токена, однако в любом случае биграммная модель не справляется с тем, чтобы выдавать достаточно осмысленные последовательности. Видно, что, в частности, на модель негативно влияет плохое "понимание" границ предложения, а также тот факт, что она "видит" только ближайший контекст слова (в нашем случае – предыдущее и последующее слово), из-за чего возникают нелогичные последовательности: например, с двойным обозначением принадлежности: *мне не скажете **свои желания лизы** опустилась.* Однако в целом такая модель способна справляться с простыми задачами в рамках небольших контекстов, например, может использоваться как помощник в выборе следующих слов при написании сообщения.